In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("unsloth/Llama-3.2-1B", legacy=False)

In [2]:
import duckdb
from tqdm import tqdm

# Conectar ou criar o banco de dados DuckDB
conn = duckdb.connect('train_dataset2.duckdb')

In [3]:
import torch
import gc

if not torch.cuda.is_available():
    print("CUDA is not available. Please ensure you have a compatible GPU and drivers installed.")
else:
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")

Using GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [4]:
from transformers import AutoModelForCausalLM

# Recarrega o modelo salvo
model = AutoModelForCausalLM.from_pretrained('./Llama-3.2-1b-Gather/').to('cpu')

# Verifica a estrutura do modelo recarregado
print(model)

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at ./Llama-3.2-1b-Gather/ and are newly initialized: ['model.norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList()
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): LlamaRotaryEmbedding()
  )
  (lm_head): Linear(in_features=2048, out_features=128256, bias=False)
)


In [5]:
def text_to_embedding(input_text):
    with torch.no_grad():
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cpu')
        embedding = model.model.embed_tokens(input_ids) # [1,2048] até [1024,2048]
        embedding = embedding.cpu()  # Move para CPU
    return embedding[0]

In [6]:
import time
import logging
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

#logging.basicConfig(level=logging.INFO)
#logger = logging.getLogger(__name__)

class SequenceDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        #start_time = time.time()
        
        row = self.df.iloc[idx]
        sequence1 = text_to_embedding(row['content1'])
        sequence2 = text_to_embedding(row['content2'])
        
        # Use the transformed target
        target = torch.tensor(row['target'], dtype=torch.float32)
        
        # Convert sequences to tensors (no need to reconvert embedding)
        sequence1 = sequence1.float()
        sequence2 = sequence2.float()
        
        #end_time = time.time()
        #if idx % 1000 == 0:
        #    logger.info(f"Processing sample {idx}/{len(self.df)} - Time: {end_time - start_time:.4f}s")
        
        return sequence1, sequence2, target

In [7]:
df_training = conn.execute("""
WITH random_indices AS (
  SELECT id, indice, content
  FROM dataset
  WHERE type = 1
  ORDER BY RANDOM()
)
SELECT
  a.indice AS indice1,
  b.indice AS indice2,
  a.content AS content1,
  b.content AS content2,
  CASE 
    WHEN a.indice > b.indice THEN 0
    ELSE 1
  END AS target
FROM random_indices a
JOIN random_indices b
  ON a.id != b.id
ORDER BY RANDOM()
LIMIT 64000;
""").df()

# Compute the index difference
df_training

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,indice1,indice2,content1,content2,target
0,212566,475011,lado o capitalismo aparecerá aos,hugo chávez em 27 de maio de 2007 e teve sua s...,1
1,447730,1075483,grupos globalistas bilionários os metacapitali...,índio e um metalúrgico podem chegar à presidên...,1
2,835853,531647,matemática que usou os instrumentos da sua dis...,o candidato e sabe a que veio é essa a condiçã...,0
3,1084589,219455,e função no plano geral da,atrasar têm sentidos ambíguos que se alternam ...,0
4,1112381,438128,pânico na casa branca tirar o sono do papa ou ...,um progresso da democracia 23,0
...,...,...,...,...,...
63995,1037805,819313,homossexual vem num tom de quem advogasse medi...,que importa não é conhecer,0
63996,247199,742582,mataram acidentalmente trinta civis,necessitará largar na ociosidade uma boa parce...,1
63997,651390,592042,brasil os discursos revolucionários,quanto incompreensíveis e condenando,0
63998,427763,415440,a transformar seus órgãos de mídia em megafone...,dessa regra vem alcançando sucesso é que enqua...,0


In [8]:
data = {
    'target': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'content1': [
        'A tempestade durou várias horas durante a noite.',
        'Ele estudou intensamente para o exame.',
        'A fábrica reduziu a emissão de poluentes.',
        'Maria começou a praticar exercícios regularmente.',
        'O trânsito intenso durante a manhã atrasou o ônibus.',
        'A empresa investiu em tecnologia de ponta.',
        'A empresa adotou um sistema de trabalho remoto.',
        'A escola implementou aulas de reforço após o expediente.',
        'A falta de irrigação adequada prejudicou as plantações.',
        'João começou a acordar mais cedo todos os dias.',
        'A falta de manutenção no sistema elétrico causou um curto-circuito.',
        'Ana adotou uma alimentação mais saudável e balanceada.',
        'A biblioteca expandiu seu acervo com novos livros e recursos digitais.',
        'Carlos decidiu aprender a programar em Python.',
        'A chuva intensa causou inundações nas áreas baixas.',
        'Paula participou de um curso de liderança.',
        'O avião enfrentou uma forte turbulência durante o voo.',
        'A cidade implementou ciclovias em várias áreas.'
    ],
    'content2': [
        'Pela manhã, muitas ruas estavam alagadas.',
        'Conseguir uma nota alta na prova.',
        'A qualidade do ar na cidade melhorou significativamente.',
        'Ela perdeu peso e aumentou sua energia.',
        'Muitas pessoas chegaram atrasadas ao trabalho.',
        'A produção aumentou e os custos diminuíram.',
        'Os funcionários tiveram maior flexibilidade e a produtividade aumentou.',
        'O desempenho acadêmico dos alunos melhorou significativamente.',
        'A colheita deste ano foi muito menor do que o esperado.',
        'Ele conseguiu concluir suas tarefas antes do horário de trabalho.',
        'Houve um apagão que afetou toda a vizinhança por várias horas.',
        'Ela notou uma melhora significativa em sua saúde e disposição.',
        'O número de visitantes aumentou significativamente, beneficiando estudantes e pesquisadores.',
        'Ele conseguiu um emprego melhor na área de tecnologia.',
        'Muitas casas foram danificadas e moradores tiveram que ser realocados.',
        'Ela foi promovida a gerente na empresa.',
        'Muitos passageiros ficaram assustados e alguns tiveram que receber atendimento médico.',
        'Houve um aumento no número de pessoas que utilizam bicicletas para se deslocar, reduzindo o trânsito e a poluição.'
    ]
}

# Crie o DataFrame
df_validation = pd.DataFrame(data)

# Adicionar novas frases com a ordem contrária e target 0
df_reversed = pd.DataFrame({
    'target': [0] * len(data['target']),
    'content1': df_validation['content2'],
    'content2': df_validation['content1']
})

# Concatenar os dois DataFrames
df_validation = pd.concat([df_validation, df_reversed], ignore_index=True)

df_validation

,target,content1,content2
0,1,A tempestade durou várias horas durante a noite.,"Pela manhã, muitas ruas estavam alagadas."
1,1,Ele estudou intensamente para o exame.,Conseguir uma nota alta na prova.
2,1,A fábrica reduziu a emissão de poluentes.,A qualidade do ar na cidade melhorou significa...
3,1,Maria começou a praticar exercícios regularmente.,Ela perdeu peso e aumentou sua energia.
4,1,O trânsito intenso durante a manhã atrasou o ô...,Muitas pessoas chegaram atrasadas ao trabalho.
5,1,A empresa investiu em tecnologia de ponta.,A produção aumentou e os custos diminuíram.
6,1,A empresa adotou um sistema de trabalho remoto.,Os funcionários tiveram maior flexibilidade e ...
7,1,A escola implementou aulas de reforço após o e...,O desempenho acadêmico dos alunos melhorou sig...
8,1,A falta de irrigação adequada prejudicou as pl...,A colheita deste ano foi muito menor do que o ...
9,1,João começou a acordar mais cedo todos os dias.,Ele conseguiu concluir suas tarefas antes do h...


In [9]:
# Função para o collate_fn, necessária para lidar com sequências de tamanhos variáveis
def collate_fn(batch):
    sequences1, sequences2, targets = zip(*batch)
    return sequences1, sequences2, torch.stack(targets)

In [10]:
training_dataset = SequenceDataset(df_training)
validation_dataset = SequenceDataset(df_validation)

# Criar DataLoaders
train_dataloader = DataLoader(
    training_dataset, 
    batch_size=100, 
    shuffle=True, 
    num_workers=0, 
    pin_memory=False, 
    collate_fn=collate_fn
)

validation_dataloader = DataLoader(
    validation_dataset, 
    batch_size=18, 
    shuffle=False, 
    num_workers=0, 
    pin_memory=False, 
    collate_fn=collate_fn
)

In [11]:
import torch
import torch.nn as nn

class RegressionRNN(nn.Module):
    def __init__(self, input_size=2048, hidden_size=128, num_layers=2):
        super(RegressionRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = True  # Configurado como True permanentemente

        # Define os RNNs para as duas sequências com bidirecionalidade
        self.rnn1 = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True,
            bidirectional=self.bidirectional,
            dropout=0.3
        )
        
        self.rnn2 = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True,
            bidirectional=self.bidirectional,
            dropout=0.3
        )
        
        # Cada RNN bidirecional produz hidden_size * 2
        # Como temos duas sequências, o tamanho final é hidden_size * 4
        final_hidden_size = hidden_size * 4  # (hidden_size * 2) * 2
        
        # Primeira camada totalmente conectada
        self.fc1 = nn.Linear(final_hidden_size, 32)
        self.relu = nn.ReLU()
        
        # Segunda camada totalmente conectada para a saída
        self.fc2 = nn.Linear(32, 1)
    
    def _process_sequences(self, sequences, rnn):
        """
        Função auxiliar para processar uma lista de sequências através de um RNN.

        Args:
            sequences (list of tensors): Lista de tensores de entrada com tamanhos variados.
            rnn (nn.LSTM): RNN a ser usado para processar as sequências.

        Returns:
            Tensor: Estado oculto concatenado de todas as sequências no batch.
        """
        # Calcula os comprimentos das sequências
        lengths = torch.tensor([seq.size(0) for seq in sequences], dtype=torch.long, device=sequences[0].device)
        
        # Padroniza as sequências para o mesmo comprimento
        padded_sequences = nn.utils.rnn.pad_sequence(sequences, batch_first=True)
        
        # Empacota as sequências padronizadas
        packed_input = nn.utils.rnn.pack_padded_sequence(
            padded_sequences, 
            lengths.cpu(), 
            batch_first=True, 
            enforce_sorted=False
        )
        
        # Passa pelas camadas RNN
        packed_output, (h_n, _) = rnn(packed_input)
        
        # h_n tem a forma (num_layers * num_directions, batch, hidden_size)
        # Para bidirectional=True, num_directions = 2
        # Extraímos os estados ocultos das últimas camadas forward e backward
        h_n_forward = h_n[-2, :, :]  # Última camada forward: forma (batch, hidden_size)
        h_n_backward = h_n[-1, :, :]  # Última camada backward: forma (batch, hidden_size)
        
        # Concatena os estados ocultos forward e backward
        combined = torch.cat((h_n_forward, h_n_backward), dim=1)  # forma: (batch, hidden_size * 2)
        
        return combined
    
    def forward(self, sequences1, sequences2):
        """
        Forward pass do modelo.

        Args:
            sequences1 (list of tensors): Primeira lista de sequências de entrada.
            sequences2 (list of tensors): Segunda lista de sequências de entrada.

        Returns:
            Tensor: Saída de regressão para cada par de sequências.
        """
        # Processa ambas as sequências usando a função auxiliar
        last_output1 = self._process_sequences(sequences1, self.rnn1)  # Forma: (batch, hidden_size * 2)
        last_output2 = self._process_sequences(sequences2, self.rnn2)  # Forma: (batch, hidden_size * 2)
        
        # Concatena as saídas das duas sequências
        combined_output = torch.cat((last_output1, last_output2), dim=1)  # Forma: (batch, hidden_size * 4)
        
        # Passa pela primeira camada totalmente conectada
        out = self.fc1(combined_output)  # Forma: (batch, 128)
        out = self.relu(out)             # Aplicação da função de ativação
        
        # Passa pela segunda camada totalmente conectada
        out = self.fc2(out)              # Forma: (batch, 1)
        
        return out.squeeze(1)  # Forma: (batch,)

In [12]:
# Inicializa o modelo, critério de perda e otimizador
model_order = RegressionRNN().to('cuda')
#criterion = nn.MSELoss() 
criterion = nn.BCEWithLogitsLoss()

In [13]:
optimizer = torch.optim.Adam(model_order.parameters(), lr=0.001, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.1, patience=3, verbose=True
)

In [14]:
# Inicializar o contador de passos
step = 0

In [ ]:
num_epochs = 20

In [16]:
from torch.utils.tensorboard import SummaryWriter

2024-11-10 01:32:53.062429: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 01:32:53.462424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 01:32:53.727571: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 01:32:53.754287: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 01:32:54.002511: I tensorflow/core/platform/cpu_feature_guar

In [17]:
# Inicializar o writer do TensorBoard
writer = SummaryWriter('runs/treinamento_regressao_rnn')

epoch_progress = tqdm(range(num_epochs), desc='Treinamento', unit='época')
for epoch in epoch_progress:
    epoch_progress.set_description(f'Época {epoch+1}/{num_epochs}')
    
    # Treinamento
    model_order.train()
    train_loss = 0.0
    for sequences1_batch, sequences2_batch, targets_batch in train_dataloader:
        optimizer.zero_grad()
        
        # Transferir dados para GPU
        sequences1_batch = [seq.to('cuda') for seq in sequences1_batch]
        sequences2_batch = [seq.to('cuda') for seq in sequences2_batch]
        targets_batch = targets_batch.to('cuda')
        
        # Forward pass
        outputs = model_order(sequences1_batch, sequences2_batch)
        loss = criterion(outputs, targets_batch)
        
        # Backward pass e otimização
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        
        # Registrar a perda de treinamento a cada passo
        writer.add_scalar('Perda_Treinamento_Passo', loss.item(), step)
        step += 1
    
    avg_train_loss = train_loss / len(train_dataloader)
    
    # Validação
    model_order.eval()
    validation_loss = 0.0
    
    with torch.no_grad():
        for sequences1_batch, sequences2_batch, targets_batch in validation_dataloader:
            sequences1_batch = [seq.to('cuda') for seq in sequences1_batch]
            sequences2_batch = [seq.to('cuda') for seq in sequences2_batch]
            targets_batch = targets_batch.to('cuda')
            
            # Forward pass sem autocast
            outputs = model_order(sequences1_batch, sequences2_batch)
            loss = criterion(outputs, targets_batch)
            validation_loss += loss.item()
    
    avg_validation_loss = validation_loss / len(validation_dataloader)
    
    # Registrar as métricas de validação por época
    writer.add_scalar('Perda_Validação_Epoca', avg_validation_loss, epoch+1)
    
    # Passar a perda de validação para o scheduler
    scheduler.step(avg_validation_loss)
    
    # Registrar a taxa de aprendizado atual (opcional)
    current_lr = optimizer.param_groups[0]['lr']
    writer.add_scalar('Taxa_Aprendizado', current_lr, epoch+1)
    
    # Atualizar a descrição da barra com as perdas atuais
    epoch_progress.set_postfix({'Perda Treinamento': f'{avg_train_loss:.4f}', 
                                 'Perda Validação': f'{avg_validation_loss:.4f}',
                                 'LR': f'{current_lr:.6f}'})
    
    # Liberar cache da GPU após cada época
    torch.cuda.empty_cache()
    
# Fechar o SummaryWriter ao final do treinamento
writer.close()

Época 480/1000:  48%|████▊     | 479/1000 [9:38:19<10:29:02, 72.44s/época, Perda Treinamento=0.3250, Perda Validação=1.2216, LR=0.000000]


KeyboardInterrupt: 

In [21]:
model_order(text_to_embedding("bom dia").to('cuda'),text_to_embedding("boa noite").to('cuda'))

/home/jadson/anaconda3/lib/python3.12/site-packages/torch/nn/modules/rnn.py:920: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /opt/conda/conda-bld/pytorch_1720538439675/work/aten/src/ATen/native/cudnn/RNN.cpp:1424.)
  result = _VF.lstm(input, batch_sizes, hx, self._flat_weights, self.bias,


RuntimeError: shape '[4194304, 1]' is invalid for input of size 1048576

In [24]:
text_to_embedding("bom dia").shape

torch.Size([4, 2048])

In [18]:
import os

# Diretório onde os modelos serão salvos
save_dir = './saved_models'
os.makedirs(save_dir, exist_ok=True)

# Salvando o estado do modelo
model_path = os.path.join(save_dir, 'regression_rnn_model.pth')
torch.save(model_order.state_dict(), model_path)
print(f"Modelo salvo em {model_path}")

# Opcional: Salvando o estado do otimizador
optimizer_path = os.path.join(save_dir, 'optimizer_state.pth')
torch.save(optimizer.state_dict(), optimizer_path)
print(f"Estado do otimizador salvo em {optimizer_path}")

# Opcional: Salvar a época atual para retomar o treinamento
epoch_path = os.path.join(save_dir, 'last_epoch.pth')
torch.save({'epoch': epoch, 'model_state_dict': model_order.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_validation_loss}, epoch_path)
print(f"Checkpoint salvo em {epoch_path}")

Modelo salvo em ./saved_models/regression_rnn_model.pth
Estado do otimizador salvo em ./saved_models/optimizer_state.pth
Checkpoint salvo em ./saved_models/last_epoch.pth


In [ ]:
del model_order  # Remove a referência ao modelo

# Libera a memória da GPU, se o modelo estiver usando CUDA
torch.cuda.empty_cache()

# Força a coleta de lixo
gc.collect()

2268